# Can Guelph Feed Itself?

If civilization collapses, can Guelph feed itself within its own border?

Sources:
- https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
- https://www150.statcan.gc.ca/n1/en/catalogue/982600032021001
- https://www12.statcan.gc.ca/census-recensement/2021/geo/maps-cartes/referencemaps-cartesdereference/index2021-eng.cfm

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# read census of provinces and territories into a small, meaningful dataset
useful_columns = ['DGUID', 'CENSUS_YEAR', 'GEO_LEVEL', 'GEO_NAME', 'CHARACTERISTIC_ID',
       'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL']
#useful_characteristic_ids = [1, 2, 6, 7, 1522, 1523, 1526, 1527, 1528, 1529, 1537]
useful_characteristic_ids = [1, 2, 6, 7]

KCAL_PER_PERSON_DAY = 2000
PROTEIN_G_PER_PERSON_DAY = 35
DAYS_PER_YEAR = 365

In [ ]:
# read hierarchy of territorial divisions

# Reference guide for Dissemination Geographics Relationship File
# https://www150.statcan.gc.ca/n1/pub/98-26-0003/982600032021001-eng.htm

# Province and Territory, Census Division, Census Subdivision, Economic Region
hierarchy_useful_columns = ["PRDGUID_PRIDUGD", "CDDGUID_DRIDUGD", "CSDDGUID_SDRIDUGD", "ERDGUID_REIDUGD"]

# 2021 Census – Dissemination Geographies Relationship File
# https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/dguid-idugd/index2021-eng.cfm?year=21
hierarchy_df = pd.read_csv('2021_98260004.csv', usecols=hierarchy_useful_columns).drop_duplicates()

er_map_df = hierarchy_df[["PRDGUID_PRIDUGD", "ERDGUID_REIDUGD"]].drop_duplicates()

cd_map_df = hierarchy_df[["PRDGUID_PRIDUGD", "ERDGUID_REIDUGD", "CDDGUID_DRIDUGD"]].drop_duplicates()

csd_map_df = hierarchy_df[["PRDGUID_PRIDUGD", "ERDGUID_REIDUGD", "CDDGUID_DRIDUGD", "CSDDGUID_SDRIDUGD"]].drop_duplicates()

In [ ]:
# 2021 Census - Reference Maps
# https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
# 2021 Census - Standard Geographical Classification (SGC) reference maps
def get_ontario_province_census_df():

    df = pd.read_csv('./ca_provinces_census/98-401-X2021001_English_CSV_data.csv', encoding='utf-8', encoding_errors='replace', usecols=useful_columns)
    
    # filter only by useful characteristic ids
    on_df = df[df['CHARACTERISTIC_ID'].isin(useful_characteristic_ids) & (df['GEO_NAME'] == 'Ontario')]
    
    del df

    return on_df

In [ ]:
# https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
def get_ontario_census_division_census_df(cd_map_df, on_df):

    # read census of census divisions into a small, meaningful dataset
    useful_columns = ['DGUID', 'CENSUS_YEAR', 'GEO_LEVEL', 'GEO_NAME', 'CHARACTERISTIC_ID',
           'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL']
    
    df = pd.read_csv('./ca_cd_census/98-401-X2021004_English_CSV_data.csv', encoding='utf-8', encoding_errors='replace', usecols=useful_columns)
    
    #useful_characteristic_ids = [1, 2, 6, 7, 1522, 1523, 1526, 1527, 1528, 1529, 1537]
    useful_characteristic_ids = [1, 2, 6, 7]
    
    ca_cd_df = df[df['CHARACTERISTIC_ID'].isin(useful_characteristic_ids)]
    
    del df

    on_province_dguid = on_df["DGUID"].unique()[0]

    on_cd_dguids = cd_map_df.loc[cd_map_df["PRDGUID_PRIDUGD"] == on_province_dguid, 'CDDGUID_DRIDUGD'].values

    on_cd_df = ca_cd_df[ca_cd_df["DGUID"].isin(on_cd_dguids)]

    return on_cd_df

In [ ]:
# https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
def get_on_census_subdivision_census_df():
    # read census of census subdivisions in BC into a small, meaningful dataset
    
    df = pd.read_csv('./ca_on_csd_census/98-401-X2021021_English_CSV_data.csv', encoding='utf-8', encoding_errors='replace', usecols=useful_columns)
    
    on_csd_df = df[df['CHARACTERISTIC_ID'].isin(useful_characteristic_ids)]
    
    del df

    return on_csd_df

In [ ]:
# https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
def get_on_economic_regions_census_df(er_map_df, on_province_df):

    df = pd.read_csv('./ca_er_census/98-401-X2021008_English_CSV_data.csv', encoding='utf-8', encoding_errors='replace', usecols=useful_columns)
    
    ca_er_df = df[df['CHARACTERISTIC_ID'].isin(useful_characteristic_ids)]
    
    del df

    on_province_dguid = on_province_df["DGUID"].unique()[0]

    on_cd_dguids = er_map_df.loc[er_map_df["PRDGUID_PRIDUGD"] == on_province_dguid, 'ERDGUID_REIDUGD'].values

    on_er_df = ca_er_df[ca_er_df["DGUID"].isin(on_cd_dguids)]

    return on_er_df

In [ ]:
on_df = get_ontario_province_census_df()

# all of ontario census divisions
on_cd_df = get_ontario_census_division_census_df(cd_map_df, on_df)

# all of ontario census subdivisions
on_csd_df = get_on_census_subdivision_census_df()

In [ ]:
# now filter for the guelph region
wellington_county_cd_df = on_cd_df.loc[on_cd_df["GEO_NAME"] == "Wellington, County (CTY)", :]

wellington_county_dguid = on_cd_df.loc[on_cd_df["GEO_NAME"] == "Wellington, County (CTY)", "DGUID"].values[0]

# all of ontario census subdivisions
wellington_county_csd_dguids = csd_map_df.loc[csd_map_df["CDDGUID_DRIDUGD"] == wellington_county_dguid, 'CSDDGUID_SDRIDUGD'].values

wellington_county_csd_df = on_csd_df[on_csd_df["DGUID"].isin(wellington_county_csd_dguids)]

guelph_df = wellington_county_csd_df[wellington_county_csd_df["GEO_NAME"] == "Guelph, City (CY)"]

In [ ]:
on_er_df = get_on_economic_regions_census_df(er_map_df, on_df)

wellington_er_dguid = cd_map_df.loc[cd_map_df["CDDGUID_DRIDUGD"] == wellington_county_dguid, "ERDGUID_REIDUGD"].values[0]

In [ ]:
on_er_df[on_er_df["DGUID"] == wellington_er_dguid]

In [ ]:
#wellington_county_cd_df

In [ ]:
guelph_df

In [ ]:
#wellington_county_csd_df

In [ ]:
guelph_population = guelph_df.loc[guelph_df["CHARACTERISTIC_NAME"] == "Population, 2021", "C1_COUNT_TOTAL"].values[0]
guelph_land_area_m2 = guelph_df.loc[guelph_df["CHARACTERISTIC_NAME"] == "Land area in square kilometres", "C1_COUNT_TOTAL"].values[0] * 1e6

In [ ]:
guelph_population

In [ ]:
guelph_land_area_m2

In [ ]:
# get the farm yields in ontario

farm_yields_ontario_df = pd.read_csv('farm_yields_ontario_df.csv', index_col=0)
farm_yields_sask_df = pd.read_csv('farm_yields_sask_df.csv', index_col=0)

farm_yields_df = pd.concat([farm_yields_ontario_df, farm_yields_sask_df]).drop_duplicates('crop_name')

In [ ]:
farm_yields_df

In [ ]:
food_calories_df = pd.read_csv('food_calories.csv', index_col=0)
food_protein_df = pd.read_csv('food_protein.csv', index_col=0)

In [ ]:
# now, get an average calorie and protein value for each of these food groups

# potato
potato_calorie_df = food_calories_df[
    food_calories_df["FoodDescription"].str.contains('potato', case=False)
    & (
        food_calories_df["FoodDescription"].str.contains('raw', case=False)
        | food_calories_df["FoodDescription"].str.contains('boiled', case=False)
    )
    & ~food_calories_df["FoodDescription"].str.contains('leaves', case=False)
].groupby(by="FoodDescription").first().reset_index()

potato_kcal_per_kg = potato_calorie_df["NutrientValue100g"].mean() * 10

potato_protein_df = food_protein_df[
    food_protein_df["FoodDescription"].str.contains('potato', case=False)
    & (
        food_protein_df["FoodDescription"].str.contains('raw', case=False)
        | food_protein_df["FoodDescription"].str.contains('boiled', case=False)
    )
    & ~food_protein_df["FoodDescription"].str.contains('leaves', case=False)
].groupby(by="FoodDescription").first().reset_index()

potato_g_protein_per_kg = potato_protein_df["NutrientValue100g"].mean() * 10

In [ ]:
# soybeans
soybeans_calorie_df = food_calories_df[
    food_calories_df["FoodDescription"].str.contains('tempe', case=False)
    | food_calories_df["FoodDescription"].str.contains('natto', case=False)
].groupby(by="FoodDescription").first().reset_index()

soybean_kcal_per_kg = soybeans_calorie_df["NutrientValue100g"].mean() * 10

soybeans_protein_df = food_protein_df[
    food_calories_df["FoodDescription"].str.contains('tempe', case=False)
    | food_calories_df["FoodDescription"].str.contains('natto', case=False)
].groupby(by="FoodDescription").first().reset_index()

soybean_g_protein_per_kg = soybeans_protein_df["NutrientValue100g"].mean() * 10

In [ ]:
# wheat
wheat_calorie_df = food_calories_df[
    ( 
        food_calories_df["FoodDescription"].str.contains('spaghetti', case=False)
        & food_calories_df["FoodDescription"].str.contains('pasta', case=False)
        & food_calories_df["FoodDescription"].str.contains('dry', case=False)
        & ~food_calories_df["FoodDescription"].str.contains('enriched', case=False)
    ) 
    | (
        food_calories_df["FoodDescription"].str.contains('wheat flour', case=False)
        & (food_calories_df["FoodGroupCode"] == 20)
        & ~food_calories_df["FoodDescription"].str.contains('enriched', case=False)
    )
].groupby(by="FoodDescription").first().reset_index()

wheat_kcal_per_kg = wheat_calorie_df["NutrientValue100g"].mean() * 10

wheat_protein_df = food_protein_df[
    (
        food_calories_df["FoodDescription"].str.contains('spaghetti', case=False)
        & food_calories_df["FoodDescription"].str.contains('pasta', case=False)
        & food_calories_df["FoodDescription"].str.contains('dry', case=False)
        & ~food_calories_df["FoodDescription"].str.contains('enriched', case=False)
    ) 
    | (
        food_calories_df["FoodDescription"].str.contains('wheat flour', case=False)
        & (food_calories_df["FoodGroupCode"] == 20)
        & ~food_calories_df["FoodDescription"].str.contains('enriched', case=False)
    )
].groupby(by="FoodDescription").first().reset_index()

wheat_g_protein_per_kg = wheat_protein_df["NutrientValue100g"].mean() * 10

In [ ]:
# oats

oats_calories_df = food_calories_df[
    food_calories_df["FoodDescription"] == 'Grains, oats'
].groupby(by="FoodDescription").first().reset_index()

oats_kcal_per_kg = oats_calories_df["NutrientValue100g"].mean() * 10

oats_proteins_df = food_protein_df[
    food_calories_df["FoodDescription"] == 'Grains, oats'
].groupby(by="FoodDescription").first().reset_index()

oats_g_protein_per_kg = oats_proteins_df["NutrientValue100g"].mean() * 10

In [ ]:
# lentils

lentil_calories_df = food_calories_df[
    food_calories_df["FoodDescription"].str.contains('lentils', case=False)
    & food_calories_df["FoodDescription"].str.contains('raw', case=False)
    & ~food_calories_df["FoodDescription"].str.contains('sprouted', case=False)
].groupby(by="FoodDescription").first().reset_index()

lentil_kcal_per_kg = lentil_calories_df["NutrientValue100g"].mean() * 10

lentil_proteins_df = food_protein_df[
    food_calories_df["FoodDescription"].str.contains('lentils', case=False)
    & food_calories_df["FoodDescription"].str.contains('raw', case=False)
    & ~food_calories_df["FoodDescription"].str.contains('sprouted', case=False)
].groupby(by="FoodDescription").first().reset_index()

lentil_g_protein_per_kg = lentil_proteins_df["NutrientValue100g"].mean() * 10

In [ ]:
# rye

rye_calories_df = food_calories_df[
    food_calories_df["FoodDescription"].str.contains('rye', case=False)
].groupby(by="FoodDescription").first().reset_index()

rye_kcal_per_kg = lentil_calories_df["NutrientValue100g"].mean() * 10

rye_proteins_df = food_protein_df[
    food_calories_df["FoodDescription"].str.contains('lentils', case=False)
].groupby(by="FoodDescription").first().reset_index()

rye_g_protein_per_kg = lentil_proteins_df["NutrientValue100g"].mean() * 10

In [ ]:
# now, create a dataframe of food, kcal, g protein per kg
nutrient_dict = [
    {
        'food_name': 'Potato',
        'kcal_per_kg': potato_kcal_per_kg,
        'g_protein_per_kg': potato_g_protein_per_kg,
    },
    {
        'food_name': 'Soybeans',
        'kcal_per_kg': soybean_kcal_per_kg,
        'g_protein_per_kg': soybean_g_protein_per_kg
    },
    {
        'food_name': 'Spring Wheat',
        'kcal_per_kg': wheat_kcal_per_kg,
        'g_protein_per_kg': wheat_g_protein_per_kg
    },
    {
        'food_name': 'Winter Wheat',
        'kcal_per_kg': wheat_kcal_per_kg,
        'g_protein_per_kg': wheat_g_protein_per_kg
    },
    {
        'food_name': 'Oats',
        'kcal_per_kg': oats_kcal_per_kg,
        'g_protein_per_kg': oats_g_protein_per_kg
    },
    {
        'food_name': 'Lentil',
        'kcal_per_kg': lentil_kcal_per_kg,
        'g_protein_per_kg': lentil_g_protein_per_kg
    },
    {
        'food_name': 'Rye',
        'kcal_per_kg': rye_kcal_per_kg,
        'g_protein_per_kg': rye_g_protein_per_kg
    }
]

nutrient_df = pd.DataFrame(nutrient_dict)

# then, merge that with yield to get kcal, g protein per m2

# then, introduce another dataframe with planting ratios

# then, do a calorie and protein check to see if self-sufficiency is viable!!!

In [ ]:
nutrients_per_m2_df = (
    nutrient_df
    .merge(farm_yields_df, left_on='food_name', right_on='crop_name', how='inner')
    .drop(columns=['crop_name', 'province_state', 'region', 'crop_description'])
)

In [ ]:
nutrients_per_m2_df['kcal_per_m2'] = nutrients_per_m2_df['kcal_per_kg'] * nutrients_per_m2_df['yield_kg_per_m2']
nutrients_per_m2_df['g_protein_per_m2'] = nutrients_per_m2_df['g_protein_per_kg'] * nutrients_per_m2_df['yield_kg_per_m2']

In [ ]:
nutrients_per_m2_df

In [ ]:
# now, create the planning chart
percentage_of_city_to_plant = 0.56
ground_coverage_efficiency_of_planting = 0.8

planting_ratios_dict = [
    {
        'food_name': 'Potato',
        'planting_proportion': 0
    },
    {
        'food_name': 'Soybeans',
        'planting_proportion': 0.3
    },
    {
        'food_name': 'Spring Wheat',
        'planting_proportion': 0.2
    },
    {
        'food_name': 'Winter Wheat',
        'planting_proportion': 0.8
    },
    {
        'food_name': 'Oats',
        'planting_proportion': 0.5
    },
    {
        'food_name': 'Lentil',
        'planting_proportion': 0
    },
    {
        'food_name': "Rye",
        'planting_proportion': 0
    }
]

planting_ratios_df = pd.DataFrame(planting_ratios_dict)

#assert planting_ratios_df['planting_proportion'].sum() == 1

planting_ratios_df["crop_area_m2"] = (
    guelph_land_area_m2 
    * percentage_of_city_to_plant 
    * ground_coverage_efficiency_of_planting
    * planting_ratios_df["planting_proportion"]
)

In [ ]:
total_nutrients_df = nutrients_per_m2_df.merge(planting_ratios_df, on='food_name')

total_nutrients_df["total_mass_kg"] = total_nutrients_df["yield_kg_per_m2"] * total_nutrients_df["crop_area_m2"]
total_nutrients_df["total_kcal"] = total_nutrients_df["kcal_per_m2"] * total_nutrients_df["crop_area_m2"]
total_nutrients_df["total_g_protein"] = total_nutrients_df["g_protein_per_m2"] * total_nutrients_df["crop_area_m2"]

In [ ]:
total_nutrients_df

In [ ]:
total_kcal = total_nutrients_df["total_kcal"].sum()
required_kcal = DAYS_PER_YEAR * KCAL_PER_PERSON_DAY * guelph_population

total_g_protein = total_nutrients_df["total_g_protein"].sum()
required_g_protein = DAYS_PER_YEAR * PROTEIN_G_PER_PERSON_DAY * guelph_population

print(f"Total kcal produced: {total_kcal}")
print(f"Required kcal: {required_kcal}")
print(f"Percentage kcal requirement met: {round((total_kcal / required_kcal) * 100, 1)}")

print()

print(f"Total g protein produced: {total_g_protein}")
print(f"Required g protein: {required_g_protein}")
print(f"Percentage g protein requirement met: {round((total_g_protein / required_g_protein) * 100, 1)}")


In [ ]:
# create a per-person tabulation
individual_ration_person_year = total_nutrients_df[["food_name", "total_mass_kg"]]
individual_ration_person_week = total_nutrients_df[["food_name", "total_mass_kg"]]

individual_ration_person_year["total_mass_kg"] = individual_ration_person_year["total_mass_kg"] / guelph_population
individual_ration_person_week["total_mass_kg"] = individual_ration_person_week["total_mass_kg"] / guelph_population / 52

In [ ]:
individual_ration_person_year

In [ ]:
individual_ration_person_week

In [ ]:
arable_land_per_person = guelph_land_area_m2 * percentage_of_city_to_plant * ground_coverage_efficiency_of_planting / guelph_population

print(arable_land_per_person)

In [ ]:
guelph_land_area_m2 / guelph_population

In [ ]:
1e6 * guelph_population / guelph_land_area_m2